In [33]:
import nltk
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asdsf\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
questions = [
    "Привет, как дела?",
    "Какие у тебя хобби?",
    "Что ты думаешь о искусственном интеллекте?",
    "Какой твой любимый цвет?"
]

answers = [
    "Привет! У меня все отлично, спасибо!",
    "Я люблю общаться с людьми и узнавать новые вещи.",
    "Искусственный интеллект очень увлекательная тема, не так ли?",
    "Мой любимый цвет - синий."
]

In [35]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

In [36]:
def get_response(user_question):
    question_tfidf = vectorizer.transform([user_question])
    similarities = cosine_similarity(question_tfidf, X)
    index = np.argmax(similarities)
    return answers[index]



In [37]:
user_question = "Как твои дела?"
response = get_response(user_question)
print(response)

Привет! У меня все отлично, спасибо!
